# WM-02: Differentiable operators in 3D curvilinear generic coordinates

In [1]:
(* In this example differentiable operations in 3D curvilinear coordinates are computed *)
(* The results are projected to cartesian frame and then to one of the selected orthogonal ones for comparison with WM *)
(* Note, results for parametric WM charts, like "Toroidal" do not match, no adjustment for handedness is performed *)
(* Note, low case letters are used for covariant components, while upper case letters are used for contravariant ones *)

In [5]:
(* Differential operations in curvilinear coordinates *)

(* Gradient *)
(* Input  : scalar *)
(* Output : covariant vector *)

ClearAll[grad] ;
grad[sqrt_][scalar_, coordinates_] :=  D[scalar, {coordinates}] ;

(* Divergence *)
(* Input  : contravariant vector *)
(* Output : scalar *)

ClearAll[div] ;
div[sqrt_][vector_, coordinates_] := 1/sqrt*Div[sqrt*vector, coordinates] ;

(* Curl *)
(* Input  : covariant vector *)
(* Output : contravariant vector *)

ClearAll[curl] ;
curl[sqrt_][vector_, coordinates_] := 1/sqrt*Curl[vector, coordinates] ;

(* Laplacian *)
(* Input  : scalar *)
(* Output : scalar *)

ClearAll[laplacian] ;
laplacian[sqrt_, inverse_][scalar_, coordinates_] := div[sqrt][inverse . grad[sqrt][scalar, coordinates], coordinates] ;

In [26]:
(* Start with generic cartesian vector field *)

fCartesian = {fx[qx, qy, qz], fy[qx, qy, qz], fz[qx, qy, qz]} ;

(* Select build-in chart *)

chart = "Hyperspherical" ;

(* Coordinates (q1, q2, q3) are contravariant coordinates *)

coordinates = {q1, q2, q3} ;

(* Transformed field using build-in function *)

fChart = Simplify[TransformedField["Cartesian" -> chart, fCartesian, {qx, qy, qz} -> coordinates]] ;

(* Radius vector *)

radius = CoordinateTransform[chart -> "Cartesian", coordinates] ;

(* Covariant basis *)

{ea, eb, ec} = Transpose[D[radius, {coordinates}]] ;

(* Metric tensor *)

forward = Simplify[Outer[Dot, {ea, eb, ec}, {ea, eb, ec}, 1]] ;

(* Metric tensor inverse *)

inverse = Simplify[Inverse[forward]] ;

(* Scaling factor *)

sqrt = Simplify[Sqrt[Det[forward]]] ;

(* Contravariant basis *)

{eA, eB, eC} = Transpose[inverse] . {ea, eb, ec} ;

(* Transformation *)

transform = Thread[{qx, qy, qz} -> radius] ;

(* Construct covariant components *)

{fa, fb, fc} = Simplify[{ea, eb, ec} . fCartesian /. transform] ;

(* Construct contravariant components *)

{fA, fB, fC}  = Simplify[Transpose[inverse . {fa, fb, fc}] ] ;

(* Projection basis (orthonormal basis) *)

e1 = Simplify[ea/Sqrt[Total[ea^2]]] ;
e2 = Simplify[eb/Sqrt[Total[eb^2]]] ;
e3 = Simplify[ec/Sqrt[Total[ec^2]]] ;

In [56]:
(* Transformation (cartesian -> orthogonal) *)

Simplify@PowerExpand@Simplify[fChart - Simplify[Dot[{e1, e2, e3}, fCartesian]] /. transform]

{0, 0, 0}

In [58]:
(* Transformation (covariant -> contravariant -> cartesian -> orhtogonal) *)

fCovariant = {fa, fb, fc} ;
fContravariant = Transpose[inverse . fCovariant] ;
fCartesian = Simplify[Dot[fContravariant, {ea, eb, ec}]] ;
Simplify@PowerExpand@Simplify[fChart - Simplify[Dot[{e1, e2, e3}, fCartesian]] /. transform]

{0, 0, 0}

In [63]:
(* Gradient *)

fWM = Grad[f @@ coordinates, coordinates, chart] ;

fCovariant = grad[sqrt][f @@ coordinates, coordinates] ;
fContravariant = Transpose[inverse . fCovariant] ;
fCartesian = Simplify[Dot[fContravariant, {ea, eb, ec}]] ;
fProjected = Simplify[Dot[{e1, e2, e3}, fCartesian]] ;

Simplify@PowerExpand@Simplify[fWM - fProjected]

{0, 0, 0}

In [70]:
(* Divergence *)

Simplify@PowerExpand@Simplify[Div[fChart, coordinates, chart] - div[sqrt][{fA, fB, fC}, coordinates]]

0

In [72]:
(* Curl *)

fWM = Curl[fChart, coordinates, chart] ;

fContravariant = curl[sqrt][{fa, fb, fc}, coordinates] ;
fCartesian = Simplify[Dot[fContravariant, {ea, eb, ec}]] ;
fProjected = Simplify[Dot[{e1, e2, e3}, fCartesian]] ;

Simplify@PowerExpand@Simplify[fWM - fProjected]

{0, 0, 0}

In [78]:
(* Laplacian *)

Simplify@PowerExpand@Simplify[Laplacian[f @@ coordinates, coordinates, chart] - laplacian[sqrt, inverse][f @@ coordinates, coordinates]]

0